# Interpolation of the skyview



### Pseudocode

```python
import packages such as topogenesis, pyvista, numpy and scipy.interpolate
load high resolution envelope lattice
load low resolution analysis lattice

# interpolation

extract line spaces for each coordinate:
x_space = np.linspace(start, end, steps)

Interpolating_function = RegularGridInterpolation()
interpolated_values = Interpolating_function(centroids)

# construct lattice
lattice = tg.to_lattice(interpolated_values.reshape(env_lattice.shape))

## 0. Initialization

### 0.1.Importing all necessary libraries and specifying the inputs

In [ ]:
import os
import topogenesis as tg
import pyvista as pv
import numpy as np
from scipy.interpolate import RegularGridInterpolator

### 0.2. Load the high resolution envelope lattice

In [ ]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/dynamic output/voxelized_envelope_highres.csv')
env_lattice = tg.lattice_from_csv(lattice_path)
print(env_lattice.shape)

### 0.3. Load low resolution skyview lattice

In [ ]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/dynamic output/skyview_lowres.csv')
low_skyview_lattice = tg.lattice_from_csv(lattice_path)
print(low_skyview_lattice.shape)

## 1. Interpolation

### 1.1. Interpolating the low-res skyview to create the high-res skyview lattice

In [ ]:
# line spaces
x_space = np.linspace(low_skyview_lattice.minbound[0], low_skyview_lattice.maxbound[0],low_skyview_lattice.shape[0])
y_space = np.linspace(low_skyview_lattice.minbound[1], low_skyview_lattice.maxbound[1],low_skyview_lattice.shape[1])
z_space = np.linspace(low_skyview_lattice.minbound[2], low_skyview_lattice.maxbound[2],low_skyview_lattice.shape[2])

# interpolation function
interpolating_function = RegularGridInterpolator((x_space, y_space, z_space), low_skyview_lattice, bounds_error=False, fill_value=None)

# high_res lattice
full_lattice = env_lattice + 1

# sample points
sample_points = full_lattice.centroids

# interpolation
interpolated_values = interpolating_function(sample_points)

# lattice construction
skyview_lattice = tg.to_lattice(interpolated_values.reshape(env_lattice.shape), env_lattice)

# nulling the unavailable cells
skyview_lattice *= env_lattice

print(skyview_lattice.shape)

### 1.2. Visualize the high resolution interpolation of skyview

In [ ]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = skyview_lattice.shape
# The bottom left corner of the data set
grid.origin = skyview_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = skyview_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Skyview"] = skyview_lattice.flatten(order="F")  # Flatten the Lattice

# adding the meshes
# p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')
    
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])
p.add_volume(grid, cmap="coolwarm", clim=[0, 1.0],opacity=opacity, shade=True)

# plotting
p.show(use_ipyvtk=True)

### 1.3. Save the high resolution Skyview Lattice into a CSV

In [ ]:
# save the skyview latice to csv
csv_path = os.path.relpath('../data/dynamic output/skyview.csv')
skyview_lattice.to_csv(csv_path)

### Credits

In [ ]:
__author__ = "Shervin Azadi and Pirouz Nourian"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on Solar Envelope"